In [1]:
from PIL import Image
from torch.utils.data.dataset import Dataset
from scipy.misc import imread
import numpy as np
import torch
import torch.optim as optim
from core import networks

import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import copy

In [2]:
#Hyperparameters for training
mini_batch_size = 128
lambda_ = 1e-8

In [3]:
#dataload (MNIST)
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('mnist-data/', train=True, download=True,
                       transform=transforms.Compose([transforms.ToTensor(),])),
        batch_size=mini_batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('mnist-data/', train=False, transform=transforms.Compose([transforms.ToTensor(),])
                       ),
        batch_size=mini_batch_size, shuffle=True)

In [4]:
#custom regularization

import torch.nn as nn
from core.networks import BayesianNetwork
def custom_regularization(saver_net, trainer_net,mini_batch_size, lambda_, loss=None):
    
    mean_reg = 0
    sigma_reg = 0
    
    #net1, net2에서 각 레이어에 있는 mean, sigma를 이용하여 regularization 구현

    #각 모델에 module 접근
    for saver, trainer in zip(saver_net.modules(),trainer_net.modules()):
        
        #만약 BayesianNetwork 이면
        if isinstance(saver,BayesianNetwork) and  isinstance(trainer,BayesianNetwork):
            
            #Network 내부의 layer에 순차적으로 접근
            for saver_layer, trainer_layer in zip(saver.layer_arr, trainer.layer_arr):
            
            # calculate mean regularization
            
                mean_reg += lambda_*(torch.div(trainer_layer.weight_mu, saver_layer.weight_sigma)-torch.div(trainer_layer.weight_mu, trainer_layer.weight_sigma)).norm(2)
                
                
            # calculate sigma_reg regularization
            
                sigma_reg += torch.sum(torch.div(trainer_layer.weight_sigma, saver_layer.weight_sigma) - torch.log(torch.div(trainer_layer.weight_sigma, saver_layer.weight_sigma)))
            
            sigma_reg = sigma_reg/(mini_batch_size*2)
            mean_reg = mean_reg/(mini_batch_size*2)
                
#             print (mean_reg, sigma_reg) # regularization value 확인
    print ('loss')
    print (loss)
    print ()
#     print ('mean_reg')
#     print (mean_reg)
#     print ()
#     print ('sigma_reg')
#     print (sigma_reg)
#     print ()
    
    loss = loss + mean_reg + sigma_reg 

    return loss

In [5]:
def train(saver_net,trainer_net, optimizer, epoch, mini_batch_size, lambda_, DEVICE):
    trainer_net.train()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        trainer_net.zero_grad()
        loss = trainer_net.sample_elbo(data, target, mini_batch_size, DEVICE) #홍준 코딩
        loss = custom_regularization(saver_net, trainer_net, mini_batch_size, lambda_, loss)
        loss.backward()
        optimizer.step()


In [6]:
# GPU 설정
device_num = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Model Initialization
#Saver_Net : mu = 0, sigma = log(1+exp(1))
#trainer_ner : mu = [-5,5], sigma = log(1+exp([-5,+5]))
saver_net = networks.BayesianNetwork(init_type = 'zero', DEVICE = device_num).to(device_num)
trainer_net = networks.BayesianNetwork(init_type = 'random', DEVICE = device_num).to(device_num)

optimizer = optim.Adam(saver_net.parameters())
optimizer = optim.Adam(trainer_net.parameters())

for epoch in range(10):

    #0. trainet_net variance init
    
    trainer_net.variance_init() #trainer net의 variance크게 init
    trainer_net = trainer_net.to(device_num)
    
    #1. trainet_net training 하는데 regularization을 위해서 saver_net의 정보 이용
    
    train(saver_net, trainer_net, optimizer, epoch, mini_batch_size, lambda_, device_num)

    #2. 1 batch가 끝나면 saver_net에 trainet_net을 복사 (weight = mean, sigma)
    
    saver_net = copy.deepcopy(trainer_net)
    

/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


loss
tensor(78724608., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(56832376., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(73653344., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(62907692., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(55189896., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(72313552., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(76388656., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(81100216., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(70801864., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(64384516., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(57632140., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(63240252., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(61461864., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(74705536., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(76205008., device='cuda:0', grad_fn=

loss
tensor(48683396., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(53821572., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(59588044., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(68655304., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(62109884., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(56865952., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(62971860., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(72722496., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(52254780., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(57335108., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(63181508., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(71447992., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(76357056., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(62193068., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(58244264., device='cuda:0', grad_fn=

loss
tensor(51535636., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(51365088., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(46112276., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(48226632., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(61357336., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(53586588., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(55524492., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(58016920., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(60046092., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(73190216., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(58896972., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(51654212., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(45099708., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(46627760., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(65525828., device='cuda:0', grad_fn=

loss
tensor(56762424., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(39694148., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(53770136., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(40915828., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(45701292., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(56554888., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(51008556., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(47464420., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(44651372., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(49872744., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(42926532., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(62048024., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(48587028., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(48610984., device='cuda:0', grad_fn=<NllLossBackward>)

loss
tensor(51088764., device='cuda:0', grad_fn=

RuntimeError: The expanded size of the tensor (128) must match the existing size (96) at non-singleton dimension 0.  Target sizes: [128, 10].  Tensor sizes: [96, 10]